tgb - 11/13/2019 - Testing if rescaling the inputs changes the ability of the network to generalize.   
Architecture = From [], predict []   
Test 1 = Rephrase all water concentrations as RH  
Test 2 = Rephrase all temperature as deviations from moist adiabat  
  
tgb - 11/15/2019 - This notebook is mostly used for pre-processing

# 0) Imports

In [1]:
from cbrain.imports import *
from cbrain.data_generator import *
from cbrain.cam_constants import *
from cbrain.losses import *
from cbrain.utils import limit_mem
from cbrain.layers import *
import tensorflow as tf
from tensorflow import math as tfm
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
# physical_devices = tf.config.experimental.list_physical_devices('GPU') 
# tf.config.experimental.set_memory_growth(physical_devices[0], True)
# tf.config.experimental.set_memory_growth(physical_devices[1], True)
# tf.config.experimental.set_memory_growth(physical_devices[2], True)
import xarray as xr
import numpy as np
from cbrain.model_diagnostics import ModelDiagnostics
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as imag
import scipy.integrate as sin

TRAINDIR = '/local/Tom.Beucler/SPCAM_PHYS/'
DATADIR = '/project/meteo/w2w/A6/S.Rasp/SP-CAM/fluxbypass_aqua/'
PREFIX = '8col009_01_'
%cd /filer/z-sv-pool12c/t/Tom.Beucler/SPCAM/CBRAIN-CAM



/nfspool-0/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog
[Errno 2] No such file or directory: '/filer/z-sv-pool12c/t/Tom.Beucler/SPCAM/CBRAIN-CAM'
/nfspool-0/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog


# 1) Preprocessing

In [2]:
%cd /export/nfs0home/tbeucler/CBRAIN-CAM

/nfspool-0/home/tbeucler/CBRAIN-CAM


In [4]:
!python preprocessing-11132019.py -c pp_config/2022_01_10_Train_Valid_For_Nando_t-dt.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
01/10/2022 04:19:14 PM Preprocess training dataset with real_geography flag set to False
01/10/2022 04:19:14 PM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_09_02_TRAIN_For_Nando.nc
01/10/2022 04:19:14 PM Reading input files
01/10/2022 04:19:14 PM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/fluxbypass_aqua/AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0001-0[1-4]-*-*.nc
01/10/2022 04:19:17 PM Crop levels
01/10/2022 04:19:17 PM Create stacked dataarray
01/10/2022 04:19:17 PM Real geography flag set to False
01/10/2022 04:19:17 PM Aquaplanet simulation detected
var is  QBP
var is  TBP
var is  VBP
var is  PS
var is  SOLIN
var is  SHFLX
var is  LHFLX
var is  PHQ
var is  TPHYSTND
var is  FSNT
var is  FSNS
var is  FLNT
var is  FLNS
var is  PRECT
var is  PHQt-dt
var is  TPHYSTNDt-dt
var is  FSNTt-dt
var is  FSNSt-dt
var is  FLNTt-dt
var is  FLNSt-dt
var is  PRECTt-dt
These time steps are cut: []
01/10/2022 04:19:17 PM S

OMP: Info #250: KMP_AFFINITY: pid 272716 tid 273152 thread 36 bound to OS proc set 25
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 273430 thread 37 bound to OS proc set 29
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 273431 thread 38 bound to OS proc set 33
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 273154 thread 39 bound to OS proc set 35
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 273437 thread 40 bound to OS proc set 31
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 273438 thread 41 bound to OS proc set 27
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 273156 thread 42 bound to OS proc set 37
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 273447 thread 43 bound to OS proc set 41
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 273448 thread 44 bound to OS proc set 45
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 273158 thread 45 bound to OS proc set 47
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 273454 thread 46 bound to OS proc set 43
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 273455 th

OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275491 thread 139 bound to OS proc set 41
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275490 thread 138 bound to OS proc set 37
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275492 thread 140 bound to OS proc set 45
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275493 thread 141 bound to OS proc set 47
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275494 thread 142 bound to OS proc set 43
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275495 thread 143 bound to OS proc set 39
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275496 thread 144 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275497 thread 145 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275498 thread 146 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275499 thread 147 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275500 thread 148 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 27

OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275672 thread 243 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275674 thread 245 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275673 thread 244 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275675 thread 246 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275676 thread 247 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275677 thread 248 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275678 thread 249 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275680 thread 251 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275679 thread 250 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275681 thread 252 bound to OS proc set 1
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275682 thread 253 bound to OS proc set 5
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 27

OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275860 thread 350 bound to OS proc set 9
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275859 thread 349 bound to OS proc set 5
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275858 thread 348 bound to OS proc set 1
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275861 thread 351 bound to OS proc set 11
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275862 thread 352 bound to OS proc set 7
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275864 thread 354 bound to OS proc set 13
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275863 thread 353 bound to OS proc set 3
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275866 thread 356 bound to OS proc set 21
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275865 thread 355 bound to OS proc set 17
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275867 thread 357 bound to OS proc set 23
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275868 thread 358 bound to OS proc set 19
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 275

...

OMP: Info #250: KMP_AFFINITY: pid 272716 tid 276161 thread 558 bound to OS proc set 36
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 276162 thread 559 bound to OS proc set 40
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 276163 thread 560 bound to OS proc set 44
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 276164 thread 561 bound to OS proc set 46
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 276165 thread 562 bound to OS proc set 42
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 276166 thread 563 bound to OS proc set 38
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 276167 thread 564 bound to OS proc set 25
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 276168 thread 565 bound to OS proc set 29
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 276169 thread 566 bound to OS proc set 33
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 276170 thread 567 bound to OS proc set 35
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 276171 thread 568 bound to OS proc set 31
OMP: Info #250: KMP_AFFINITY: pid 272716 ti

OMP: Info #250: KMP_AFFINITY: pid 272716 tid 273171 thread 672 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 276930 thread 673 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 276932 thread 675 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 276931 thread 674 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 276933 thread 676 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 276934 thread 677 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 276935 thread 678 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 276936 thread 679 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 276937 thread 680 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 276938 thread 681 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 276939 thread 682 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 276

OMP: Info #250: KMP_AFFINITY: pid 272716 tid 273167 thread 768 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 1184 thread 769 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 1185 thread 770 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 1187 thread 772 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 1186 thread 771 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 1188 thread 773 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 1189 thread 774 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 1190 thread 775 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 1191 thread 776 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 1192 thread 777 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 1193 thread 778 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 1194 thread 779 bound t

OMP: Info #250: KMP_AFFINITY: pid 272716 tid 14654 thread 852 bound to OS proc set 25
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 14656 thread 854 bound to OS proc set 33
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 14655 thread 853 bound to OS proc set 29
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 14657 thread 855 bound to OS proc set 35
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 14658 thread 856 bound to OS proc set 31
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 14659 thread 857 bound to OS proc set 27
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 14660 thread 858 bound to OS proc set 37
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 14662 thread 860 bound to OS proc set 45
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 14663 thread 861 bound to OS proc set 47
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 14664 thread 862 bound to OS proc set 43
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 14661 thread 859 bound to OS proc set 41
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 14665 thr

OMP: Info #250: KMP_AFFINITY: pid 272716 tid 273145 thread 963 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 18827 thread 965 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 18826 thread 964 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 18828 thread 966 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 18829 thread 967 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 18830 thread 968 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 18832 thread 970 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 18831 thread 969 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 18833 thread 971 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 18834 thread 972 bound to OS proc set 1
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 18835 thread 973 bound to OS proc set 5
OMP: Info #250: KMP_AFFINITY: pid 272716 tid 18836 thread

In [5]:
!python preprocessing-11132019.py -c pp_config/2021_09_02_Train_Valid_For_Nando.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
01/10/2022 06:30:39 PM Preprocess training dataset with real_geography flag set to False
01/10/2022 06:30:39 PM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_09_02_TRAIN_For_Nando.nc
01/10/2022 06:30:39 PM Reading input files
01/10/2022 06:30:39 PM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/fluxbypass_aqua/AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0001-0[1-4]-*-*.nc
01/10/2022 06:33:56 PM Crop levels
01/10/2022 06:33:56 PM Create stacked dataarray
01/10/2022 06:33:56 PM Real geography flag set to False
01/10/2022 06:33:56 PM Aquaplanet simulation detected
var is  QBP
var is  TBP
var is  VBP
var is  PS
var is  SOLIN
var is  SHFLX
var is  LHFLX
var is  PHQ
var is  TPHYSTND
var is  FSNT
var is  FSNS
var is  FLNT
var is  FLNS
var is  PRECT
These time steps are cut: []
01/10/2022 06:33:56 PM Stack and reshape dataarray
01/10/2022 06:34:25 PM Save dataarray as /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_09

OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78598 thread 48 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78704 thread 52 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78701 thread 49 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78702 thread 50 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78703 thread 51 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78705 thread 53 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78706 thread 54 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78707 thread 55 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78708 thread 56 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78709 thread 57 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78710 thread 58 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78712 thread 60 bound to OS proc set 

OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78608 thread 147 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78868 thread 148 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78869 thread 149 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78610 thread 150 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78874 thread 152 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78873 thread 151 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78875 thread 153 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78876 thread 154 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78877 thread 155 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78878 thread 156 bound to OS proc set 1
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78879 thread 157 bound to OS proc set 5
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78880 thread 158 bound to

OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78620 thread 249 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78979 thread 251 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78978 thread 250 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78981 thread 253 bound to OS proc set 5
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78980 thread 252 bound to OS proc set 1
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78982 thread 254 bound to OS proc set 9
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78984 thread 256 bound to OS proc set 7
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78983 thread 255 bound to OS proc set 11
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78985 thread 257 bound to OS proc set 3
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78986 thread 258 bound to OS proc set 13
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78988 thread 260 bound to OS proc set 21
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78987 thread 259 bound to 

OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78630 thread 348 bound to OS proc set 1
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 79153 thread 350 bound to OS proc set 9
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 79152 thread 349 bound to OS proc set 5
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 79155 thread 352 bound to OS proc set 7
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 79154 thread 351 bound to OS proc set 11
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 79156 thread 353 bound to OS proc set 3
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 79157 thread 354 bound to OS proc set 13
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 79158 thread 355 bound to OS proc set 17
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 79160 thread 357 bound to OS proc set 23
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 79159 thread 356 bound to OS proc set 21
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 79162 thread 359 bound to OS proc set 15
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 79161 thread 358 bound to 

OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78640 thread 447 bound to OS proc set 11
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 79257 thread 448 bound to OS proc set 7
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 79258 thread 449 bound to OS proc set 3
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78599 thread 450 bound to OS proc set 13
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 79321 thread 451 bound to OS proc set 17
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 79322 thread 452 bound to OS proc set 21
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 79325 thread 455 bound to OS proc set 15
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 79323 thread 453 bound to OS proc set 23
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 79324 thread 454 bound to OS proc set 19
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 79326 thread 456 bound to OS proc set 24
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 79327 thread 457 bound to OS proc set 28
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 79329 thread 459 bound 

01/10/2022 06:48:25 PM Done!
01/10/2022 06:48:26 PM Shuffle training dataset
01/10/2022 06:48:26 PM Start shuffling /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_09_02_TRAIN_For_Nando.nc into /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_09_02_TRAIN_For_Nando_shuffle.nc. Open and create datasets.
01/10/2022 06:48:28 PM Shuffle!
100%|█████████████████████████████████████████████| 5/5 [02:00<00:00, 23.84s/it]
01/10/2022 06:50:28 PM Closing datasets
01/10/2022 06:50:28 PM Done!
01/10/2022 06:50:28 PM Preprocess validation dataset
01/10/2022 06:50:28 PM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_09_02_VALID_For_Nando.nc
01/10/2022 06:50:28 PM Reading input files
01/10/2022 06:50:28 PM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/fluxbypass_aqua/AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0001-0[5-8]-*-*.nc
01/10/2022 06:54:01 PM Crop levels
01/10/2022 06:54:01 PM Create stacked dataarray
01/10/2022 06:54:01 PM Real geography flag set t

OMP: Info #250: KMP_AFFINITY: pid 72462 tid 91561 thread 639 bound to OS proc set 11
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 91563 thread 641 bound to OS proc set 3
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 91562 thread 640 bound to OS proc set 7
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 91564 thread 642 bound to OS proc set 13
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 91565 thread 643 bound to OS proc set 17
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 91567 thread 645 bound to OS proc set 23
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 91566 thread 644 bound to OS proc set 21
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 91569 thread 647 bound to OS proc set 15
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 91568 thread 646 bound to OS proc set 19
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 91570 thread 648 bound to OS proc set 24
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 91571 thread 649 bound to OS proc set 28
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 91572 thread 650 bound 

OMP: Info #250: KMP_AFFINITY: pid 72462 tid 78621 thread 744 bound to OS proc set 24
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 93370 thread 745 bound to OS proc set 28
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 93371 thread 746 bound to OS proc set 32
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 93424 thread 747 bound to OS proc set 34
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 93425 thread 748 bound to OS proc set 30
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 93426 thread 749 bound to OS proc set 26
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 93427 thread 750 bound to OS proc set 36
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 93428 thread 751 bound to OS proc set 40
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 93429 thread 752 bound to OS proc set 44
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 93430 thread 753 bound to OS proc set 46
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 93431 thread 754 bound to OS proc set 42
OMP: Info #250: KMP_AFFINITY: pid 72462 tid 93432 thread 755 boun

In [3]:
!python preprocessing-11132019.py -c pp_config/2021_11_30_Norm_For_Nando_Cl_Inv.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
11/30/2021 07:58:34 AM Preprocess training dataset with real_geography flag set to False
11/30/2021 07:58:34 AM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_11_30_TRAIN_For_Nando_ClInv.nc
11/30/2021 07:58:34 AM Reading input files
11/30/2021 07:58:34 AM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/fluxbypass_aqua/AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0001-01-*-*.nc
11/30/2021 07:58:39 AM Crop levels
11/30/2021 07:58:39 AM Create stacked dataarray
11/30/2021 07:58:39 AM Real geography flag set to False
11/30/2021 07:58:39 AM Aquaplanet simulation detected
var is  RH
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc set respected: 0-47
OMP: Info #156: KMP_AFFINITY: 48 available OS procs
OMP: Info #157: KMP_AFFINITY: Uniform topology
OMP: Info #179: KMP_AFFINITY: 2 packages x 12 cores

OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114307 thread 48 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114470 thread 49 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114471 thread 50 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114472 thread 51 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114474 thread 53 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114473 thread 52 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114475 thread 54 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114476 thread 55 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114477 thread 56 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114478 thread 57 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114479 thread 58 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114480 thread 

OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114341 thread 144 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114609 thread 145 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114610 thread 146 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114611 thread 147 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114612 thread 148 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114613 thread 149 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114614 thread 150 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114615 thread 151 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114616 thread 152 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114617 thread 153 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114618 thread 154 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114

OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114348 thread 240 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114708 thread 242 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114707 thread 241 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114709 thread 243 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114710 thread 244 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114711 thread 245 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114712 thread 246 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114714 thread 248 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114713 thread 247 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114715 thread 249 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114716 thread 250 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114

OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114316 thread 336 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114805 thread 338 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114804 thread 337 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114806 thread 339 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114807 thread 340 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114808 thread 341 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114809 thread 342 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114810 thread 343 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114811 thread 344 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114813 thread 346 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114812 thread 345 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114

...

OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114323 thread 720 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115259 thread 721 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115260 thread 722 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115262 thread 724 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115261 thread 723 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115263 thread 725 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115264 thread 726 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115265 thread 727 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115266 thread 728 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115268 thread 730 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115269 thread 731 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115

OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114346 thread 816 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115357 thread 819 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115355 thread 817 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115356 thread 818 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115359 thread 821 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115360 thread 822 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115358 thread 820 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115362 thread 824 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115361 thread 823 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115363 thread 825 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115364 thread 826 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115

OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114310 thread 912 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115453 thread 913 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115454 thread 914 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115455 thread 915 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115456 thread 916 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115457 thread 917 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115459 thread 919 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115458 thread 918 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115460 thread 920 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115461 thread 921 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115462 thread 922 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115

OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114327 thread 1008 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115607 thread 1010 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115608 thread 1011 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115606 thread 1009 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115611 thread 1014 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115610 thread 1013 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115609 thread 1012 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115613 thread 1016 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115612 thread 1015 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115614 thread 1017 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115615 thread 1018 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 114

OMP: Info #250: KMP_AFFINITY: pid 114226 tid 114338 thread 1104 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115712 thread 1105 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115713 thread 1106 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115715 thread 1108 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115714 thread 1107 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115716 thread 1109 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115719 thread 1112 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115720 thread 1113 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115718 thread 1111 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115717 thread 1110 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 114226 tid 115722 thread 1115 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 114

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_09_02_Train_Valid_For_Nando.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_09_02_Test_for_Nando.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_09_03_Valid_for_Nando_Cl_Inv.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_09_03_Test_for_Nando_Cl_Inv.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_09_13_Nando_ClInv_small_data_test.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_06_16_T_BMSE_input_small_data_test.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_06_06_RG_Test-0K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_06_03_RG_Train_Valid-0K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_04_24_RG_PERC_Train_Valid-M4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_04_24_RG_PERC_Train_Valid-P4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_04_24_RG_PERC_Test-P4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_04_24_RG_PERC_Test-M4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_04_09_PERC_Test-P4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_04_09_PERC_Test-M4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_04_09_PERC_Train_Valid-P4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_04_09_PERC_Train_Valid.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_04_09_PERC_Test.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_03_31_O3_T_NSto220_input_small_data_test.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_03_18_O3_Train_Valid-P4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_03_18_O3_Test-P4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_03_18_O3_Train_Valid-M4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_03_18_O3_Test-M4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_02_01_O3_BCONS_input_small_data_test.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_03_01_O3_LHFnsDELQ_input_small_data_test.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_03_01_O3_LHFnsQ_input_small_data_test.yml

In [ ]:
!python preprocessing-11132019.py -c /export/nfs0home/tbeucler/CBRAIN-CAM/pp_config/2021_03_01_O3_LHFnsDELQ_input_small_data_test.yml

In [ ]:
!python preprocessing-11132019.py -c /export/nfs0home/tbeucler/CBRAIN-CAM/pp_config/2021_03_01_O3_LHFnsQ_input_small_data_test.yml

In [ ]:
!python preprocessing-11132019.py -c /export/nfs0home/tbeucler/CBRAIN-CAM/pp_config/2021_02_01_O3_QSATdeficit_input_small_data_test.yml

In [ ]:
!python preprocessing-11132019.py -c /export/nfs0home/tbeucler/CBRAIN-CAM/pp_config/2021_01_24_O3_input_small_data_test

In [ ]:
!python preprocessing-11132019.py -c /export/nfs0home/tbeucler/CBRAIN-CAM/pp_config/2021_01_24_O3_Train_Valid.yml

In [ ]:
!python preprocessing-11132019.py -c /export/nfs0home/tbeucler/CBRAIN-CAM/pp_config/2021_01_24_O3_Test.yml

In [ ]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/147_POG_RHinput_FLUXoutput_test.yml

In [ ]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/118_POG_TfromTSinput_test.yml

In [ ]:
!ls -la -h /local/Tom.Beucler/SPCAM_PHYS/1*

# 2) Check the preprocessed data

In [ ]:
path_data = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/'

In [ ]:
path_RGsmall = path_data + '2021_04_18_NORM_RG_small.nc'

In [ ]:
path_QSATdeficit = path_data+'2021_02_01_NORM_O3_QSATdeficit_small.nc'

In [ ]:
path_TfromNS = path_data+'2021_02_01_NORM_O3_TfromNS_small.nc'

In [ ]:
path_Bcons = path_data+'2021_03_17_NORM_O3_BCONS_small.nc'

In [ ]:
path_BMSE = path_data + '2021_06_16_NORM_BMSE_small.nc'

In [ ]:
data_RGsmall = xr.open_dataset(path_RGsmall)

In [ ]:
# data_QSATdeficit = xr.open_dataset(path_QSATdeficit)
# data_TfromNS = xr.open_dataset(path_TfromNS)
# data_Bcons = xr.open_dataset(path_Bcons)
data_Bmse = xr.open_dataset(path_BMSE)

In [ ]:
data_QSATdeficit

In [ ]:
data_TfromNS

In [ ]:
data_Bmse['var_names']

In [ ]:
data_TfromNS['mean'].shape

In [ ]:
data_Bmse['mean'][:30].values

In [ ]:
data_Bmse['mean'][30:60].values

In [ ]:
plt.hist(data_Bmse['mean'])

In [ ]:
data_Bcons['mean'][30:60]

In [ ]:
data_Bcons['std'][30:60]

# 3) Preprocessing for PRL paper

In [ ]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/8col_rad_tbeucler_local-RH-test.yml

In [ ]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/8col_rad_tbeucler_local-test.yml